# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». \
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали. \
Нужно построить модель с максимально большим значением accuracy, довести долю правильных ответов по крайней мере до 0.75, проверить accuracy на тестовой выборке.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## Разбейте данные на выборки

Выделим из датасета целевой столбец. Так как необходимо определить какой тариф Смарт или Ультра лучше предложить клиенту для целевого столбца подойдет is_ultra.

In [4]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']
print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


Разобьем данные на выборки для обучения(доля 3/5 от общих данных), валидации (1/5) и тестирования (1/5).

In [5]:
features_train, features_test_and_valid, target_train, target_test_and_valid = train_test_split(
    features, target, test_size=0.4)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test_and_valid, target_test_and_valid, test_size = 0.5)
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Исследуйте модели

Исследуем и выбирем лучшую модель для данной задачи.

### Исследуем модель Дерево

Для модели Дерево найдем наилучшие параметры и посчитаем метрику качества accuracy.

In [6]:
depth_tree = 0
accuracy_tree = 0
best_model_tree = None
for depth_cur in range(1, 10):
    model = DecisionTreeClassifier(max_depth=depth_cur, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy_current = accuracy_score(target_valid, predictions_valid)
    if accuracy_current > accuracy_tree:
        accuracy_tree = accuracy_current
        depth_tree = depth_cur
        best_model_tree = model
print("Accuracy наилучшей модели на валидационной выборке:", accuracy_tree, "Максимальная глубина:", depth_tree)
#print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129 Максимальная глубина: 5


### Исследуем модель Случайный Лес

Для модели Случайный Лес найдем наилучшие параметры и посчитаем метрику качества accuracy.

In [7]:
best_model_forest = None
est_forest = 0
accuracy_forest = 0
depth_forest = 0
for est in range(10, 51):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        accuracy_current = model.score(features_valid, target_valid)
        if accuracy_current > accuracy_forest:
            best_model_forest = model
            est_forest = est
            accuracy_forest = accuracy_current
            depth_forest = depth
print("Accuracy наилучшей модели на валидационной выборке:", accuracy_forest, "Количество деревьев:", est_forest, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936 Количество деревьев: 16 Максимальная глубина: 10


### Исследуем модель Логистическая Регрессия

Для модели Логическая Регрессия посчитаем метрику качества accuracy

In [8]:
model_log_reg = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100)
model_log_reg.fit(features_train, target_train)
pred = model_log_reg.predict(features_valid)
accuracy_log_reg = accuracy_score(target_valid, pred)
accuracy_log_reg

0.6702954898911353

## Проверьте модель на тестовой выборке

In [9]:
best_model_forest.score(features_test, target_test)

0.7978227060653188

## (бонус) Проверьте модели на адекватность

In [10]:
best_model_tree.score(features_test, target_test)

0.8009331259720062

In [11]:
pred = model_log_reg.predict(features_test)
accuracy_score(target_test, pred)

0.7153965785381027

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
